<a href="https://colab.research.google.com/github/Virajbiz98/Python_Learning/blob/main/test_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

!pip install gradio==4.44.0 requests==2.32.3

In [13]:

import gradio as gr
import requests
import json

In [14]:

#model_name = "mistralai/mistral-small"
#api_key = "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"
#API_URL = "https://openrouter.ai/api/v1/chat/completions"
model_name = "gemini-2.0-flash"
API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
api_key = ""

In [15]:


def apply_settings(user_api_key, chosen_model):
  global api_key, model_name
  api_key  = user_api_key or model_name

  model_name = chosen_model or model_name




In [16]:
from os import truncate
def stream_chat_reply(user_message, conversation_history=[]):
  if not api_key:
    return "Please pasete your  API KEY!"

  messages = [
      {"role": "system", "content":"You are a useful python tutor!"}

  ]


  for past_user_message, past_bot_message in conversation_history:
    if past_user_message:
      messages.append({"role": "user", "content": past_user_message})
    if past_bot_message:
      messages.append({"role": "assistant", "content": past_bot_message})

  headers = {
      "Authorization": "Bearer" + api_key,
      "Content-Type": "application/json",
      "Accept": "text/event-stream"
  }

  body = {
      "model": model_name,
      "messages": messages,
      "stream": True

  }

  with requests.post(API_URL, headers=headers, json=body, stream=True, timeout=300) as response:
    partial_test = ""

    for raw in response.iter_lines(dexode_unicode=True):
      if not raw or not raw.startswith("data:"):
        continue

      data = raw[5:].strip()
      if data == "[DONE]":
        break

      if not data.startswith("{"):
        continue


      chunk = json.loads(data)
      delta = (chunk.get("choices"))[0].get("delta")
      token = delta.get("content")
      if token:
        partial_text += token
        yield partial_text


In [17]:
from logging import PlaceHolder
with gr.Blocks() as demo:
  gr.Markdown("# Mistral Chatbot")

  with gr.Row():
    api_key_box = gr.Textbox(
        label="API KEY",
        placeholder="Paste  API KEY here",
        type="password"
    )

    model_box = gr.Dropdown(
        label="Model",
       # choices=["Mistral: Mistral Small 3.2 24B (free)", "mistralai/devstral-small-2505:free", "mistralai/mistral-7b-instruct"],
        #value="Mistral: Mistral Small 3.2 24B (free)",
        choices=["gemini-2.0-flash"],
        value="gemini-2.0-flash",
        interactive=True
    )

    apply_button = gr.Button("Apply")

  status = gr.Markdown("Past your api key, then click apply")
  apply_button.click(apply_settings, [api_key_box, model_box], [status])

  gr.Markdown("Ask me anything!")
  gr.ChatInterface(fn=stream_chat_reply).queue()


demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1d0012cc8511324124.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
